## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


##### 可以自行嘗試多種架構

### Xception 架構

In [2]:
input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer
model_x = keras.applications.xception.Xception(include_top=False, 
                                             weights='imagenet', 
                                             input_tensor=None, 
                                             input_shape=None, 
                                             pooling=None, 
                                             classes=1000)
model_x.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

## 添加層數

In [3]:
x = model_x.output

x = GlobalAveragePooling2D()(x)
x = Dense(output_dim=128, activation='relu')(x)
x = Dropout(rate = 0.1)(x)

predictions = Dense(output_dim=10, activation='softmax')(x)
model_x = Model(inputs=model_x.input, outputs=predictions)
print('Model深度：', len(model_x.layers))

Model深度： 136


## 鎖定特定幾層不要更新權重

In [4]:
for layer in model_x.layers[:100]:
    layer.trainable = False
for layer in model_x.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


## Training

In [6]:
# compile the model (should be done *after* setting layers to non-trainable)
model_x.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model_x.fit(x_train,y_train,batch_size=32,epochs=20)

Epoch 1/20
50000/50000 [==============================] - 95s 2ms/step - loss: 1.6336 - acc: 0.3981
Epoch 2/20
50000/50000 [==============================] - 90s 2ms/step - loss: 1.0754 - acc: 0.6304
Epoch 3/20
50000/50000 [==============================] - 90s 2ms/step - loss: 0.9370 - acc: 0.6784
Epoch 4/20
50000/50000 [==============================] - 90s 2ms/step - loss: 0.8465 - acc: 0.7108
Epoch 5/20
50000/50000 [==============================] - 90s 2ms/step - loss: 0.7645 - acc: 0.7364
Epoch 6/20
50000/50000 [==============================] - 91s 2ms/step - loss: 0.7006 - acc: 0.7589
Epoch 7/20
50000/50000 [==============================] - 91s 2ms/step - loss: 0.6340 - acc: 0.7810
Epoch 8/20
50000/50000 [==============================] - 91s 2ms/step - loss: 0.5727 - acc: 0.8041
Epoch 9/20
50000/50000 [==============================] - 91s 2ms/step - loss: 0.5152 - acc: 0.8212
Epoch 10/20
50000/50000 [==============================] - 90s 2ms/step - loss: 0.4612 - acc: 0.8419

### Resnet 50 架構

In [7]:
input_tensor = Input(shape=(32, 32, 3))

#include top 決定要不要加入 fully Connected Layer

model=keras.applications.ResNet50(include_top=False, 
                                  weights='imagenet', 
                                  input_tensor=input_tensor, 
                                  pooling=None, 
                                  classes=10)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [8]:
x = model.output

x = GlobalAveragePooling2D()(x)
x = Dense(output_dim=128, activation='relu')(x)
x = Dropout(rate = 0.1)(x)

predictions = Dense(output_dim=10, activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))

Model深度： 179


In [9]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

In [10]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=20)

Epoch 1/20
50000/50000 [==============================] - 171s 3ms/step - loss: 1.4049 - acc: 0.5556 
Epoch 2/20
50000/50000 [==============================] - 164s 3ms/step - loss: 1.0269 - acc: 0.6544
Epoch 3/20
50000/50000 [==============================] - 164s 3ms/step - loss: 0.8767 - acc: 0.6988
Epoch 4/20
50000/50000 [==============================] - 164s 3ms/step - loss: 0.8227 - acc: 0.7152
Epoch 5/20
50000/50000 [==============================] - 164s 3ms/step - loss: 0.7506 - acc: 0.7363
Epoch 6/20
50000/50000 [==============================] - 164s 3ms/step - loss: 0.6530 - acc: 0.7677
Epoch 7/20
50000/50000 [==============================] - 164s 3ms/step - loss: 0.6599 - acc: 0.7714
Epoch 8/20
50000/50000 [==============================] - 164s 3ms/step - loss: 0.5511 - acc: 0.8040
Epoch 9/20
50000/50000 [==============================] - 164s 3ms/step - loss: 0.5003 - acc: 0.8182
Epoch 10/20
50000/50000 [==============================] - 164s 3ms/step - loss: 0.4310 - 